In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

In [ ]:

window = tk.Tk()
window.title("Face Recognition System")
window.config(background="#137b3e")


style = ttk.Style()


style.theme_use("clam")


style.configure("TButton", font=("Consolas", 20), padding=10)
style.configure("TLabel", font=("Consolas", 20))


input_frame = ttk.Frame(window, padding="10")
input_frame.grid(column=0, row=0, sticky="nsew")


l1 = ttk.Label(input_frame, text="Name:")
l1.grid(column=0, row=0, padx=5, pady=5)
t1 = ttk.Entry(input_frame, width=50)
t1.grid(column=1, row=0, padx=5, pady=5)

l2 = ttk.Label(input_frame, text="Age:")
l2.grid(column=0, row=1, padx=5, pady=5)
t2 = ttk.Entry(input_frame, width=50)
t2.grid(column=1, row=1, padx=5, pady=5)

l3 = ttk.Label(input_frame, text="PRN No.:")
l3.grid(column=0, row=2, padx=5, pady=5)
t3 = ttk.Entry(input_frame, width=50)
t3.grid(column=1, row=2, padx=5, pady=5)

# Create a frame for the buttons
button_frame = ttk.Frame(window, padding="10")
button_frame.grid(column=0, row=1, sticky="nsew")

#TRAINING THE CLASSIFIER
def train_classifier():
    data_dir="C:/Users/91773/Desktop/face recogniser/data"
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L');
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)

    #Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result','Training Dataset Completed!!!')
    
    
b1 = ttk.Button(button_frame, text="TRAINING", command=train_classifier)
b1.grid(column=0, row=0, padx=5, pady=5)

#DETECTING THE FACE

def detect_face(): 
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
        coords = []
    
        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            id, pred = clf.predict(gray_image[y:y + h, x:x + w])
            confidence = int(100 * (1 - pred / 300))

            mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                passwd="",
                database="Authorized_user"
            )
            mycursor = mydb.cursor()
            mycursor.execute("Select Name from my_table where id=" + str(id))
            s = mycursor.fetchone()
            s = '' + ''.join(s)
    
            if confidence > 82:
                cv2.putText(img, s, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else: 
                cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
    
            coords = [x, y, w, h]
        return coords

    def recognize(img, clf, faceCascade):
        coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img
        
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    
    video_capture = cv2.VideoCapture(0)
    
    # Set the window size to 400x600
    window_width = 300
    window_height = 300
    cv2.namedWindow("Face Detection", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Face Detection", window_width, window_height)
    
    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade)
        cv2.imshow("Face Detection", img)
    
        if cv2.waitKey(1) == 13:  # Enter key to exit
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

b2 = ttk.Button(button_frame, text="DETECT THE FACE", command=detect_face)
b2.grid(column=1, row=0, padx=5, pady=5)


#GENERATING THE DATASET
def generate_dataset():
    if (t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result','Please Provide Complete Details Of The User')
    else:
        mydb=mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="",
            
        database="Authorized_user"
        )
        mycursor=mydb.cursor()
        mycursor.execute("SELECT * from my_table")
        myresult=mycursor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql = "INSERT INTO my_table (id, Name, Age, PRN) VALUES (%s, %s, %s, %s)"
        val = (id, t1.get(), t2.get(), t3.get())

        mycursor.execute(sql,val)
        mydb.commit()
        
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
            if len(faces) == 0:  # Change is to == and check if the list is empty
                return None
            for (x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
                return cropped_face
    
        cap = cv2.VideoCapture(0)
        id = 1
        img_id = 0
    
        while True:
            ret, frame = cap.read()
    
            if not ret:
                print("Failed to capture image")
                break
    
            cropped_face = face_cropped(frame)
            if cropped_face is not None:
                img_id += 1
                face = cv2.resize(cropped_face, (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Cropped face", face)
    
                if cv2.waitKey(1) == 13 or int(img_id) == 200:
                    break
    
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result','Generating Dataset Completed!!!')


b3 = ttk.Button(button_frame, text="GENERATE DATASET", command=generate_dataset)
b3.grid(column=2, row=0, padx=5, pady=5)


window.geometry("800x280")
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\91773\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\91773\AppData\Local\Temp\ipykernel_19120\913293548.py", line 116, in detect_face
    img = recognize(img, clf, faceCascade)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91773\AppData\Local\Temp\ipykernel_19120\913293548.py", line 99, in recognize
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\91773\AppData\Local\Temp\ipykernel_19120\913293548.py", line 88, in draw_boundary
    s = '' + ''.join(s)
             ^^^^^^^^^^
TypeError: can only join an iterable
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\91773\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.fun